In [1]:
#!/usr/bin/env python

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms

import time
import os
import copy
import random

import os
import pandas as pd
from torchvision.io import read_image

from torch.utils import data
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor


from torch.utils.data import DataLoader, TensorDataset, random_split, WeightedRandomSampler


from torch.nn import Conv2d
from torch import optim

import pretrained_microscopy_models as pmm
import torch.utils.model_zoo as model_zoo

torch.backends.cudnn.deterministic = True
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)


# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True


# In[2]:


import pandas as pd
import pickle



with open("Data/MoS2_Analysis_Processed_Data2", "rb") as fp:   # Unpickling
  MoS2_data = pickle.load(fp)

df = pd.DataFrame(MoS2_data)
T = df['T']

T_classes = {900.0: 0, 950.0: 1, 1000.0: 2}
#print(MoS2_activities_substrate2['T'])
T_target = [T_classes[T[index]] for index, item in enumerate(T)]
sampleId = df['sampleId']
sampleLabel = df['sampleLabel']
data_image = np.array([np.array(item) for item in df['image']])


Data_CNN = data_image
print(Data_CNN.shape)
Data_CNN_rgb = np.repeat(Data_CNN[..., np.newaxis], 3, -1)
print(Data_CNN_rgb.shape)
Data_CNN_rgb = Data_CNN_rgb.transpose(0, 3, 1, 2)/255
print(Data_CNN_rgb.shape)


from sklearn.model_selection import train_test_split

transform = transforms.Compose([
      transforms.ToPILImage(),
      transforms.RandomRotation(degrees= (0, 180)),
      transforms.RandomHorizontalFlip(0.5),
      transforms.RandomVerticalFlip(0.5),
      transforms.ToTensor(),
      #transforms.Normalize(mean=mean, std=std),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

transform_test = transforms.Compose([
      transforms.ToPILImage(),
      transforms.ToTensor(),
      #transforms.Normalize(mean=mean, std=std),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

class Dataset():
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels, transform):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.transform = transform

  def __len__(self):
        'Denotes the total number of samples' 
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = self.transform(ID)
        y = self.labels[index]
        return X, y      



batch_size = 8

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

def data_loader_fn(x, y, transform, batch_size):
    target = np.array(y)
    data = np.array(x)
    labels_unique, class_sample_count = np.unique(target, return_counts=True)
    weight = [sum(class_sample_count) / c for c in class_sample_count]


    samples_weight = np.array([weight[t] for t in target])
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
    target = torch.from_numpy(target)
    data = torch.from_numpy(data)
    #train_dataset = torch.utils.data.TensorDataset(data, target)

    dataset = Dataset(data, target, transform)
    
    data_loader = DataLoader(
        dataset, batch_size=batch_size, num_workers=1, sampler=sampler, drop_last=False)
    
    return data_loader


def data_loader_test_fn(x, y, transform, batch_size):
    data = torch.tensor(x)
    target = torch.tensor(y)
    dataset = Dataset(data, target, transform)

    data_loader = DataLoader(dataset,  batch_size = batch_size, shuffle = False, drop_last=False)#, num_workers= 2)
    return data_loader

def accuracy_cnn_fn(trained_model, data_loader, data_type):
    correct = 0
    total = 0
    trained_model.eval()
    #with torch.no_grad():
    for data in data_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = trained_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the {total} {data_type} images: {accuracy :.1f} %')
    
    return accuracy
    

def pretrained_model_image(drop_out):
    
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

    model.fc = nn.Sequential(nn.ReLU(),
                                   nn.Dropout(p=drop_out),
                                    nn.Linear(512, 100), #150, 50176
                                     #nn.ReLU(),
                                     nn.Dropout(p=drop_out),
                                     #nn.Linear(200, 100),
                                     nn.ReLU(),
                                     #nn.Dropout(p=drop_out),
                                     nn.Linear(100, 3)
                                     )
    model.to(device)
    return model

def pretrained_model_micro(drop_out):
    
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)
    url = pmm.util.get_pretrained_microscopynet_url('resnet18', 'micronet')
    map_location=torch.device('cpu')
    model.load_state_dict(model_zoo.load_url(url))


    model.fc = nn.Sequential(nn.ReLU(),
                                   nn.Dropout(p=drop_out),
                                    nn.Linear(512, 100), #150, 50176
                                     #nn.ReLU(),
                                     nn.Dropout(p=drop_out),
                                     #nn.Linear(200, 100),
                                     nn.ReLU(),
                                     #nn.Dropout(p=drop_out),
                                     nn.Linear(100, 3)
                                     )
    model.to(device)
    return model


(262, 224, 224)
(262, 224, 224, 3)
(262, 3, 224, 224)


In [2]:
with open("Model/Final_MoS2_ImageNet_best_variables", "rb") as fp:   # Unpickling
    best_variables = pickle.load(fp)


fold = 9
drop_out = best_variables[fold]['drop_out']
learning_rate = best_variables[fold]['learning_rate']
batch_size = best_variables[fold]['batch_size']
epochs = best_variables[fold]['epochs']
print(drop_out)

model_image = pretrained_model_image(drop_out)
print(model_image)

PATH = f'Model/Final_MoS2_ImageNet_{fold}_class_T.pth'

model_image.load_state_dict(torch.load(PATH))
model_image.eval()
model_image.type(torch.FloatTensor)

0.35
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tru

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
with open("Model/Final_MoS2_MicroNet_best_variables", "rb") as fp:   # Unpickling
    best_variables = pickle.load(fp)


fold = 7
drop_out = best_variables[fold]['drop_out']
learning_rate = best_variables[fold]['learning_rate']
batch_size = best_variables[fold]['batch_size']
epochs = best_variables[fold]['epochs']
print(drop_out)

model_micro = pretrained_model_image(drop_out)
print(model_micro)

PATH = f'Model/Final_MoS2_MicroNet_{fold}_class_T.pth'

model_micro.load_state_dict(torch.load(PATH))
model_micro.eval()
model_micro.type(torch.FloatTensor)

0.0
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load pre-trained ResNet model

# Choose a specific layer to extract features from
def get_features_from_model(model, images):
    model.eval()
    # Choose a specific layer to extract features from
    target_layer = model.fc[2]  # can be layer1 - layer4, avgpool

    # Define a hook to capture the activations from the target layer

    activations = []


    def hook_fn(module, input, output):
        activations.append(output)



    hook = target_layer.register_forward_hook(hook_fn)

    # Load and preprocess an image


    #def get_features(images):
    for image in images:
   
        item_tensor = torch.tensor(image, dtype=torch.float32)
        input_batch = item_tensor.unsqueeze(0)     
    # Pass the input through the model to trigger the hook and capture activations

        with torch.no_grad():

            _ = model(input_batch)


    # Remove the hook to prevent memory leaks

    hook.remove()
 

# The activations list now contains the convolutional activations from the target layer
    return activations

# Use 




In [6]:
activations = get_features_from_model(model_image, Data_CNN_rgb)
print(activations[0].shape)
print(len(activations))
flattened_ImageNet = np.array([a.numpy() for a in activations]).reshape(len(activations),-1)
print(flattened_ImageNet.shape)

torch.Size([1, 100])
262
(262, 100)


In [7]:
activations = get_features_from_model(model_micro, Data_CNN_rgb)
print(activations[0].shape)
print(len(activations))
flattened_MicroNet = np.array([a.numpy() for a in activations]).reshape(len(activations),-1)
print(flattened_MicroNet.shape)

torch.Size([1, 100])
262
(262, 100)


In [8]:
MoS2_Class_Data = []
for index, sample in enumerate(sampleId):
    this_item = {}

    this_item['sampleId'] = sample
    this_item['sampleLabel'] = sampleLabel[index]
    this_item['image'] = data_image[index]
    this_item['T'] = T[index]
    #this_item['substrate'] = substrate[index2]
    this_item['ImageNet'] = flattened_ImageNet[index]
    this_item['MicroNet'] = flattened_MicroNet[index]

    MoS2_Class_Data.append(this_item)

    #break
             
             
             
             
             
print(len(MoS2_Class_Data))  

with open("Data/MoS2_Analysis_Data_trained2", "wb") as fp:   #Pickling
  pickle.dump(MoS2_Class_Data, fp)  

262


# Augmented Data

In [8]:
import pandas as pd
import pickle
#MoS2_data_aug_imagenet_cnn

with open("Data/MoS2_Analysis_Augmented_Data", "rb") as fp:   # Unpickling
  MoS2_data_aug_imagenet_cnn = pickle.load(fp)

df = pd.DataFrame(MoS2_data_aug_imagenet_cnn)
aug_images = np.array(df['image'].tolist())
#aug_images = np.array([item for item in images])
Temp_target = df['T']
T_target = np.array(Temp_target)
sampleId = np.array(df['sampleId'].tolist())

print(aug_images.shape)

(707, 3, 224, 224)


In [6]:
activations = get_features_from_model(model_image, aug_images)
print(activations[0].shape)
print(len(activations))
flattened_aug_ImageNet = np.array([a.numpy() for a in activations]).reshape(len(activations),-1)
print(flattened_aug_ImageNet.shape)

torch.Size([1, 100])
707
(707, 100)


In [7]:
activations = get_features_from_model(model_micro, aug_images)
print(activations[0].shape)
print(len(activations))
flattened_aug_MicroNet = np.array([a.numpy() for a in activations]).reshape(len(activations),-1)
print(flattened_aug_MicroNet.shape)

torch.Size([1, 100])
707
(707, 100)


In [9]:
MoS2_Class_Data = []



for index, sample in enumerate(sampleId):
    this_item = {}

    this_item['sampleId'] = sample
   
    this_item['image'] = aug_images[index]
    this_item['T'] = Temp_target[index]
   
    this_item['ImageNet'] = flattened_aug_ImageNet[index]
    this_item['MicroNet'] = flattened_aug_MicroNet[index]

    MoS2_Class_Data.append(this_item)


             
print(len(MoS2_Class_Data))  

with open("Data/MoS2_Analysis_Augmented_trained", "wb") as fp:   #Pickling
  pickle.dump(MoS2_Class_Data, fp)  

707


# Cropped Augmentation

In [10]:
#MoS2_balanced_data_temp

with open("Data/MoS2_Analysis_Cropped_Data", "rb") as fp:   # Unpickling
  MoS2_balanced_data_temp = pickle.load(fp)

df_Temp = pd.DataFrame(MoS2_balanced_data_temp)
sampleId = np.array(df_Temp['sampleId'])
T_target = df_Temp['T'].tolist()#[item for item in df_Temp['T']]
#T_target = np.array(T_target).reshape(len(T_target),1)
images_cropped = np.array(df_Temp['image'].tolist())#np.array([np.array(item) for item in df_Temp['image']])


print(type(T_target))

Data_CNN = images_cropped
print(Data_CNN.shape)
Data_CNN_rgb = np.repeat(Data_CNN[..., np.newaxis], 3, -1)
print(Data_CNN_rgb.shape)
cropped_images = Data_CNN_rgb.transpose(0, 3, 1, 2)/255
print('cropped_images.shape', cropped_images.shape)


<class 'list'>
(699, 224, 224)
(699, 224, 224, 3)
cropped_images.shape (699, 3, 224, 224)


In [11]:

activations = get_features_from_model(model_micro, cropped_images)
print(activations[0].shape)
print(len(activations))
flattened_MicroNet_cropped = np.array([a.numpy() for a in activations]).reshape(len(activations),-1)
print(flattened_MicroNet_cropped.shape)

torch.Size([1, 100])
699
(699, 100)


In [12]:
activations = get_features_from_model(model_image, cropped_images)
print(activations[0].shape)
print(len(activations))
flattened_ImageNet_cropped = np.array([a.numpy() for a in activations]).reshape(len(activations),-1)
print(flattened_ImageNet_cropped.shape)

torch.Size([1, 100])
699
(699, 100)


In [13]:
MoS2_Class_Data = []



for index, sample in enumerate(sampleId):
    this_item = {}

    this_item['sampleId'] = sample
   
    this_item['image'] = cropped_images[index]
    this_item['T'] = T_target[index]
   
    this_item['ImageNet'] = flattened_ImageNet_cropped[index]
    this_item['MicroNet'] = flattened_MicroNet_cropped[index]

    MoS2_Class_Data.append(this_item)


             
print(len(MoS2_Class_Data))  

with open("Data/MoS2_Analysis_Cropped_trained", "wb") as fp:   #Pickling
  pickle.dump(MoS2_Class_Data, fp)  

699
